<a href="https://colab.research.google.com/github/invinciblebansal/DL-Guide/blob/main/Fashion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading the current verion of keras tuner

In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 55.8 MB/s 


Importing the necessary libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

checking its version to be 2.8.2

In [ ]:
print(tf.__version__)

2.8.2


downloading the dataset from keras datasets itself

In [ ]:
fashion=keras.datasets.fashion_mnist

loading the trsining and testing dataset from keras.datasets.fasion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion.load_data()

normalisation for better efficiency of model

In [ ]:
train_images=train_images/255
test_images=test_images/255

checking the shape of images

In [ ]:
train_images[0].shape


(28, 28)

reshaping the images

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

#Building the MODEL:

In [ ]:
def build_model(hp):
    model=keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('conv_1_filter',min_value=64,max_value=128,step=16),
            kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
            activation='relu',
            input_shape=(28,28,1)
        ),
        keras.layers.Conv2D(
            filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
            kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
            activation='relu'
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
            activation='relu'
        ),
        keras.layers.Dense(10,activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model
        
    


importing the hyperparameters to check the best model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective="val_accuracy",
                          max_trials=5,
                          directory='output',
                          project_name='FASHion')

In [ ]:
tuner_search.search(train_images,train_labels,epochs=10,validation_split=0.1)

getting the best model based on search 

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        153648    
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 64)                1486912   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,642,490
Trainable params: 1,642,490
Non-trainable params: 0
_________________________________________________________________


training our model on the best parameters found:

In [ ]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1890 - accuracy: 0.9284 - val_loss: 0.2517 - val_accuracy: 0.9128
Epoch 5/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.1418 - accuracy: 0.9474 - val_loss: 0.2657 - val_accuracy: 0.9127
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1059 - accuracy: 0.9601 - val_loss: 0.3000 - val_accuracy: 0.9097
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0769 - accuracy: 0.9711 - val_loss: 0.3270 - val_accuracy: 0.9105
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0575 - accuracy: 0.9785 - val_loss: 0.3824 - val_accuracy: 0.9137
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0434 - accuracy: 0.9844 - val_loss: 0.4234 - val_accuracy: 0.9137
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0344 - accuracy: 0.9871 - val_loss: 0.4665 - val_accurac